# Template

This notebook allows users to find common and specific genes in their experiment of interest using an *existing* VAE model (model trained by the user using `1_train_example.pynb`) and selecting a template experiment that is included in the training compendium.

This notebook will generate a `generic_gene_summary_<experiment id>.tsv` file that contains z-scores per gene that indicates how specific a gene is the experiment in question.

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

In [2]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from ponyo import utils, simulate_expression_data
from sophie import (
    process,
    stats,
    ranking,
)

/home/alexandra/anaconda3/envs/sophie/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
Using TensorFlow backend.


## Inputs

User needs to fill in the [config file](config_new_experiment.tsv) following the instructions provided in the [readme]](README.md)

In [3]:
# Read in config variables
config_filename = "config_example.tsv"

params = utils.read_config(config_filename)

In [4]:
# Load config params

# Root directory containing analysis subdirectories and scripts
base_dir = params["base_dir"]

# Local directory to store intermediate files
local_dir = params["local_dir"]

# Normalized compendium filename
normalized_compendium_filename = params["normalized_compendium_filename"]

# Metadata file that maps experiment ids to sample ids
metadata_filename = params["metadata_filename"]

# Delimiter used in metadata file
metadata_delimiter = params["metadata_delimiter"]

# Column header corresponding to experiment id and sample id
metadata_experiment_colname = params["metadata_experiment_colname"]
metadata_sample_colname = params["metadata_sample_colname"]

# ID for template experiment to be selected
project_id = params["project_id"]

# Number of simulated experiments to generate
num_simulated = params["num_simulated"]

# Directory that simulated experiments will be written to
# This directory is created by 
simulated_data_dir = params["simulated_data_dir"]

# Directory containing trained VAE model
vae_model_dir = params["vae_model_dir"]

# Size of the latent dimension
latent_dim = params["latent_dim"]

# Scaler transform used to scale compendium data into 0-1 range for training
scaler_transform_filename = params["scaler_transform_filename"]

# Which DE method to use
# We recommend that if data is RNA-seq then use DESeq2 ("deseq")
# If data is microarray then use Limma ("limma")
de_method = params["DE_method"]

# If using DE-seq, setting this parameter will
# remove genes below a certain threshold
count_threshold = params["count_threshold"]

# Metadata file that specifies which samples to keep for DE analysis (Optional)
# By default, a two-condition differential expression analysis is supported (case vs control).
# However, some experiments included more than 2 conditions and so these "extra" samples 
# should not considered in the downstream differential expression analysis. 
template_process_samples_filename = params["template_process_samples_filename"]
    
# Metadata file that specifies sample grouping for DE analysis
template_DE_grouping_filename = params["template_DE_grouping_filename"]

# Statistic to use to rank genes or pathways by
# Choices are "log2FoldChange" if using DESeq or "log2FC" 
# if using limma as the `de_method`
col_to_rank_genes = params["rank_genes_by"]

In [5]:
# Files generated by this notebook

# File storing normalized template experiment
normalized_template_filename = params["normalized_template_filename"]

# File storing un-normalized template experiment for us in downstream DE analysis
raw_template_filename = params["raw_template_filename"]

# File storing processed template experiment,
# after samples have been selected for comparison in DE analysis
processed_template_filename = params["processed_template_filename"]

# Output summary file
output_filename = params["output_filename"]

## Get unnormalized template experiment

In [7]:
process.fetch_template_experiment(
    normalized_compendium_filename,
    metadata_filename,
    metadata_delimiter,
    metadata_experiment_colname,
    project_id,
    metadata_sample_colname,
    scaler_transform_filename,
    raw_template_filename,
    normalized_template_filename
)

## Simulate data

In [8]:
# Run simulation
simulate_expression_data.shift_template_experiment(
    normalized_compendium_filename,
    vae_model_dir,
    latent_dim,
    scaler_transform_filename,
    metadata_filename,
    metadata_delimiter,
    metadata_experiment_colname,
    metadata_sample_colname,
    project_id,
    local_dir,
    simulated_data_dir,
    num_simulated)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.



## Process template and simulated experiments

* Remove samples not required for comparison
* Make sure ordering of samples matches metadata for proper comparison
* Make sure values are cast as integers if using DESeq
* Filter lowly expressed genes if using DESeq

In [9]:
# Update simulated dir
if not os.path.exists(template_process_samples_filename):
    template_process_samples_filename = None

if de_method == "deseq":
    # Process template data
    stats.process_samples_for_DESeq(
        raw_template_filename,
        template_DE_grouping_filename,
        processed_template_filename,
        count_threshold,
        template_process_samples_filename,
    )

    # Process simulated data
    for i in range(num_simulated):
        simulated_filename = os.path.join(
            simulated_data_dir,
            f"selected_simulated_data_{project_id}_{i}.tsv",
        )
        out_simulated_filename = os.path.join(
            simulated_data_dir,
            f"selected_simulated_data_{project_id}_{i}_processed.tsv",
        )
        stats.process_samples_for_DESeq(
            simulated_filename,
            template_DE_grouping_filename,
            out_simulated_filename,
            count_threshold,
            template_process_samples_filename,
        )
else:
    stats.process_samples_for_limma(
        raw_template_filename,
        template_DE_grouping_filename,
        processed_template_filename,
        template_process_samples_filename,
    )

    for i in range(num_simulated):
        simulated_filename = os.path.join(
            simulated_data_dir,
            f"selected_simulated_data_{project_id}_{i}.tsv",
        )
        stats.process_samples_for_limma(
            simulated_filename,
            template_DE_grouping_filename,
            None,
            template_process_samples_filename,
        )

sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly


## Differential expression analysis

In [10]:
# Create subdirectory: "<local_dir>/DE_stats/"
os.makedirs(os.path.join(local_dir, "DE_stats"), exist_ok=True)

In [11]:
%%R -i template_DE_grouping_filename -i project_id -i processed_template_filename -i local_dir -i base_dir -i de_method

source(paste0(base_dir, '/sophie/DE_analysis.R'))

# File created: "<local_dir>/DE_stats/DE_stats_template_data_<project_id>_real.txt"
if (de_method == "deseq"){
    get_DE_stats_DESeq(
        template_DE_grouping_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}
else{
    get_DE_stats_limma(
        template_DE_grouping_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}

R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following object is masked from ‘package:limma’:

    plotMA


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply

[1] "Checking sample ordering..."
[1] TRUE


In [12]:
%%R -i template_DE_grouping_filename -i project_id -i base_dir -i simulated_data_dir -i num_simulated -i de_method

source(paste0(base_dir, '/sophie/DE_analysis.R'))

# Files created: "<local_dir>/DE_stats/DE_stats_simulated_data_<project_id>_<n>.txt"
for (i in 0:(num_simulated-1)){
    simulated_data_filename <- paste(
        simulated_data_dir,
        "/selected_simulated_data_",
        project_id,
        "_",
        i,
        "_processed.tsv",
        sep = ""
    )
    if (de_method == "deseq"){
        get_DE_stats_DESeq(
            template_DE_grouping_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
    }
    else {
        get_DE_stats_limma(
            template_DE_grouping_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
        }
    }

[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE


## Rank genes

Genes are ranked by their "generic-ness" - how frequently these genes are changed across the simulated experiments using user-specific test statistic provided in the `col_to_rank_genes` params (i.e. log2 fold change).

In [13]:
analysis_type = "DE"
template_DE_stats_filename = os.path.join(
    local_dir, "DE_stats", f"DE_stats_template_data_{project_id}_real.txt"
)

# Added
if de_method == "deseq":
    logFC_name = "log2FoldChange"
    pvalue_name = "padj"
else:
    logFC_name = "logFC"
    pvalue_name = "adj.P.Val"

template_DE_stats, simulated_DE_summary_stats = ranking.process_and_rank_genes_pathways(
    template_DE_stats_filename,
    local_dir,
    num_simulated,
    project_id,
    analysis_type,
    col_to_rank_genes,
    logFC_name,
    pvalue_name,
)

/home/alexandra/anaconda3/envs/sophie/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


## Summary table

* Gene ID: Gene identifier (hgnc symbols for human data or PA number for *P. aeruginosa* data)
* (Real): Statistics for template experiment
* (Simulated): Statistics across simulated experiments
* Number of experiments: Number of simulated experiments
* Z-score: High z-score indicates that gene is more changed in template compared to the null set of simulated experiments (high z-score = highly specific to template experiment). These z-scores are true standard scores using mean and standard deviation. The calculation for the z-score for a given gene is
    
$$
\frac{\text{log}_2 \text{fold change of the gene in the template experiment} - mean(\text{log}_2 \text{fold change of the gene in simulated experiments)}}{variance(\text{log}_2 \text{fold change of the gene in simulated experiments)}}
$$

The range of this z-score will vary depending on the number of simulated experiments, so the number of simulated experiments should be held constant if the user is performing multiple SOPHIE runs or if they're comparing to previous SOPHIE runs performed by someone else.

* Percentile (simulated): percentile rank of the median(abs(log fold change)). So its the median absolute change for that gene across the 25 simulated experiments that is then converted to a percentile rank from 0 - 100. Where a higher percentile indicates that the gene was highly changed frequently and would suggest that the gene is more commonly DE.
* Percent DE (simulated): the fraction of the simulated experiments in which that gene was found to be DE using (log fold change > 1 and adjusted p-value < 0.05). _Note:_ you may find that many genes have a 0 fraction. This is because there is some compression that happens when pushing data through the VAE so the variance of the simulated experiments is lower compared to the real experiment. We are aware of this limitation in the VAE and are looking at how to improve the variance and biological signal captured by the VAE, however we were still able to demonstrate that for now the VAE is able to simulate realistic looking biological experiments in our previous [paper](https://academic.oup.com/gigascience/article/9/11/giaa117/5952607).


**Note:**
* If using DESeq, genes with NaN in only the `Adj P-value (Real)` column are those genes flagged because of the `cooksCutoff` parameter. The cook's distance as a diagnostic to tell if a single sample has a count which has a disproportionate impact on the log fold change and p-values. These genes are flagged with an NA in the pvalue and padj columns of the result table.

* If using DESeq with count threshold, some genes may not be present in all simulated experiments (i.e. the `Number of experiments (simulated)` will not equal the number of simulated experiments you specified in the beginning. This pre-filtering will lead to some genes found in few simulated experiments and so the background/null set for that gene is not robust. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

* If using DESeq without count threshold, some genes may still not be present in all simulated experiments (i.e. the `Number of experiments (simulated)`  will not equal the number of simulated experiments you specified in the beginning. If the gene is 0 expressed across all samples and thus automatically given an NA in `log fold change, adjusted p-value` columns. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

For more information you can read [DESeq FAQs](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#pvaluesNA)

In [14]:
# Get summary table
summary_gene_ranks = ranking.generate_summary_table(
    template_DE_stats_filename,
    template_DE_stats,
    simulated_DE_summary_stats,
    col_to_rank_genes,
    local_dir,
    "gene",
    params,
)

summary_gene_ranks.sort_values(by="Z score", ascending=False).head(10)

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
PDGFC,PDGFC,1.557450e-16,13898.0,1.106324,1.106324,9.717978e-01,4.0,0.016927,0.0,0.003088,0.000049,2,22328.578497
ADAMTS1,ADAMTS1,2.383714e-27,16559.0,2.411720,-2.411720,3.964739e-02,10975.0,61.919540,0.0,0.316862,0.000126,2,16589.256307
OXLD1,OXLD1,5.254345e-06,10582.0,0.589786,0.589786,3.078360e-01,1687.0,9.513062,0.0,0.062311,0.000033,2,15746.116326
HECTD1,HECTD1,1.998254e-02,5524.0,0.252558,0.252558,5.788700e-02,2811.0,15.855104,0.0,0.086601,0.000015,2,11387.111234
NRAP,NRAP,5.629708e-20,17505.0,5.814516,5.814516,7.525858e-02,12807.0,72.256390,0.0,0.424521,0.000476,2,11322.096216
OTULIN,OTULIN,1.532890e-05,11960.0,0.744484,0.744484,6.493801e-01,248.0,1.393669,0.0,0.022006,0.000090,2,8042.154203
HDDC3,HDDC3,2.072972e-02,6137.0,0.279293,0.279293,8.335446e-01,71.0,0.394967,0.0,0.011876,0.000035,2,7659.954540
C1orf52,C1orf52,7.047887e-03,8636.0,0.430215,-0.430215,7.199858e-06,4774.0,26.931106,0.0,0.128218,0.000042,2,7152.497447
DSEL,DSEL,3.089021e-02,9428.0,0.490626,-0.490626,1.287343e-01,5843.0,32.962817,0.0,0.151340,0.000065,2,5225.200555
GPR19,GPR19,4.228441e-03,14260.0,1.211835,1.211835,3.135018e-08,15177.0,85.628844,0.0,0.677228,0.000121,2,4435.393972


In [15]:
summary_gene_ranks.isna().any()

Gene ID                                 False
Adj P-value (Real)                       True
Rank (Real)                              True
abs(log2FoldChange) (Real)               True
log2FoldChange (Real)                    True
Median adj p-value (simulated)           True
Rank (simulated)                         True
Percentile (simulated)                   True
Percent DE (simulated)                  False
Mean abs(log2FoldChange) (simulated)     True
Std deviation (simulated)                True
Number of experiments (simulated)       False
Z score                                  True
dtype: bool

In [16]:
summary_gene_ranks[summary_gene_ranks.isna().any(axis=1)]

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
LINC01656,LINC01656,0.377342,15893.0,1.912057,-1.912057,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN
LINC01282,LINC01282,0.550875,14186.0,1.190288,1.190288,5.574358e-01,14680.0,82.824578,0.0,0.607615,NaN,1,NaN
CRYGB,CRYGB,0.824493,13366.5,0.982607,0.982607,4.861042e-02,17610.0,99.356768,1.0,1.747272,NaN,1,NaN
LINC01526,LINC01526,0.404476,12706.0,0.861461,0.861461,9.013088e-01,8081.0,45.590476,0.0,0.211918,NaN,1,NaN
TEX13A,TEX13A,0.871036,11910.5,0.742160,0.742160,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN
CASC6,CASC6,0.955264,5779.5,0.261267,0.261267,9.938764e-01,402.0,2.262597,0.0,0.028539,NaN,1,NaN
OR14J1,OR14J1,0.955264,5760.5,0.261266,0.261266,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN
LINC00408,LINC00408,0.955264,5725.5,0.261266,0.261266,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN
LINC00424,LINC00424,0.955264,5725.5,0.261266,0.261266,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN
LINC00845,LINC00845,0.955264,5725.5,0.261266,0.261266,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN


In [17]:
# Save
summary_gene_ranks.to_csv(output_filename, sep="\t")